In [1]:
# Dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import requests

In [2]:
# Set up executable path
executable_path = {"executable_path": "chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=False)

# NASA Mars News

In [3]:
# Set up url
nasa_url = "https://mars.nasa.gov/news/"

# Opens the website via pop-up browser 
browser.visit(nasa_url)

In [4]:
# Retrieve HTML of the browser 
html = browser.html

# Parse HTML with Beautiful Soup 
soup = BeautifulSoup(html, "html.parser")
results = soup.select_one("ul.item_list li.slide")

In [5]:
# Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text.
news_title = results.find("div", class_="content_title").text
news_p = results.find("div", class_="article_teaser_body").text

print(news_title)
print(news_p)

NASA's Perseverance Rover Gives High-Definition Panoramic View of Landing Site
A 360-degree panorama taken by the rover’s Mastcam-Z instrument will be discussed during a public video chat this Thursday.


# JPL Mars Space Images - Featured Image

In [6]:
# Set up url
jpl_url_root = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/"
jpl_url = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html"

# Opens the website via pop-up browser 
browser.visit(jpl_url)

In [7]:
# Click full image and more info to get image url
full_image_button = browser.links.find_by_partial_text('FULL IMAGE')
full_image_button.click()

In [8]:
# Retrieve HTML of the browser 
image_html = browser.html

# Parse HTML with Beautiful Soup 
img_soup = BeautifulSoup(image_html, 'html.parser')

In [9]:
# Find the image url
full_image_url = img_soup.select_one('img.fancybox-image').get("src")
featured_image_url = jpl_url_root + full_image_url
featured_image_url

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars3.jpg'

# Mars Facts

In [10]:
# Visit the Mars Facts webpage and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
facts_df = pd.read_html("https://space-facts.com/mars/")[0]
facts_df.columns = ["Description", "Mars"]
facts_df.set_index("Description", inplace = True)
facts_df

,Mars
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [11]:
# Use Pandas to convert the data to a HTML table string.
facts_df.to_html("facts.html")

# Mars Hemispheres

In [12]:
# Set up url
usgs_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

# Opens the website via pop-up browser 
browser.visit(usgs_url)

In [13]:
hemisphere_image_urls = []

# Get a list of all of the hemispheres
links = browser.find_by_css("a.product-item h3")

for i in range(len(links)):
    hemisphere = {}
    
    # Find the elements on each loop to avoid a stale element exception
    browser.find_by_css("a.product-item h3")[i].click()

    # Find the Sample image anchor tag and extract the href
    sample_elem = browser.links.find_by_text('Sample').first
    hemisphere['img_url'] = sample_elem['href']
    
    # Get title 
    hemisphere["title"] = browser.find_by_css("h2.title").text
    
    # Append the dictionary with the image url string and the hemisphere title to a list. 
    hemisphere_image_urls.append(hemisphere) 
    browser.back()

In [14]:
# Result
hemisphere_image_urls

[{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [15]:
# Close the browser after scraping
browser.quit()